In [443]:
import pandas as pd

In [444]:
df = pd.read_csv("Data_120294_2023-03-29.csv")


/var/folders/k_/917q0lsn2g199rm9y4s8jy5h0000gn/T/ipykernel_98216/922264029.py:1: DtypeWarning: Columns (2,64,91,94,97,106,108,109,110,111,112,114,116,119,120,122,127,139,142,143,144,145,152,153,154,155,156,157,159,160,161,163,164,165,169,170,171,172,175,178,179,180,187,189,215,216,217,218,219,317,318) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data_120294_2023-03-29.csv")


## Cleaning
- Remove patients that died in waiting list

In [445]:
print("Patients pre-cleaning: ", df.shape[0])
print("Deaths in waiting list: ", df['death_date'].notnull().sum())

# Drop all patients with death date
# df = df.drop(df[df['death_date'].notnull()].index)      #   WL DATE OF DEATH/POST 10/25/99 REMOVALS
df = df.drop(df[df['cod_wl'].notnull()].index)          #   WL Candidate Cause of Death for Death Removal
df = df.drop(df[df['cod_ostxt_wl'].notnull()].index)    #   WL Candidate Cause of Death for Death Removal, Other Specified

# print abo unique values
# print(df['abo'].unique())
print("Patients post-cleaning: ", df.shape[0])

Patients pre-cleaning:  120264
Deaths in waiting list:  8648
Patients post-cleaning:  114272


## Transforming
- Drop redundant columns

In [446]:
desired_columns = ["init_age", "gender", "abo", "init_bmi_calc", "init_date", "end_date"] # add "diab", "pramr", "prapk" ?

df = df[desired_columns]

# compute waiting list time based on init_date and end_date
df['wl_time'] = (pd.to_datetime(df['end_date'], format='%d%b%Y') - pd.to_datetime(df['init_date'], format='%d%b%Y')).dt.days

# drop init_date and end_date
df = df.drop(['init_date', 'end_date'], axis=1)

# remove any rows with NaN or Inf values
df = df.dropna()
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

print("patients: ", df.shape[0])
print(df.head())

patients:  109475
   init_age gender abo  init_bmi_calc  wl_time
0      61.0      M   O      21.133678        7
1      48.0      F   O      26.912532      631
2      66.0      M   A      34.185537      578
3      66.0      M   O      28.461720        8
4      52.0      M   O      29.760838      356


## Encode

In [447]:
# encode categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

le = LabelEncoder() 
df["abo"] = le.fit_transform(df["abo"])
df["gender"] = le.fit_transform(df["gender"])
#df["alcohol_heavy_don"] = le.fit_transform(df["alcohol_heavy_don"])
# we want to predict if it took more or less than 60 days to get a transplant

df['wl_time'] = df['wl_time'].apply(lambda x: 1 if x > 30*6 else 0)

#for col in df.columns:
#    if df[col].dtype == 'object':
#        df[col] = le.fit_transform(df[col])


# scale init_age 
scaler = StandardScaler() # use MinMaxScaler() for 0-1 scaling ?
df['init_age'] = scaler.fit_transform(df['init_age'].values.reshape(-1, 1))
df['init_bmi_calc'] = scaler.fit_transform(df['init_bmi_calc'].values.reshape(-1, 1))
#df['pramr'] = scaler.fit_transform(df['pramr'].values.reshape(-1, 1))
#df['prapk'] = scaler.fit_transform(df['prapk'].values.reshape(-1, 1))


[7 0 6 2 5 1 3 4]


KeyError: 'pramr'

## Build model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense, BatchNormalization, Dropout


X_train, X_test, y_train, y_test = train_test_split(df.drop('wl_time', axis=1), df['wl_time'], test_size=0.2, random_state=42)

"""model = Sequential([
    Dense(128, activation='relu', input_shape=(3,)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='linear')
])"""


model = Sequential([
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15, batch_size=1, verbose=1)

y_pred = model.predict(X_test)



Epoch 1/15
 5803/23245 [======>.......................] - ETA: 11s - loss: 0.6272 - accuracy: 0.6833

KeyboardInterrupt: 